Blood culture test prediction model using medical history

In [1]:
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 13.1 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16506 sha256=d951b08dad2de042bf0fc5d8ffd826867fbdd8b1bacce1eaa8e027a922f552be
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


In [2]:
import swifter

In [4]:
import pandas as pd
import keras
import numpy as np
from tensorflow.keras.utils import pad_sequences

In [5]:
import tensorflow as tf
print(tf.__version__)

2.15.0


Uploading Medical Data

In [21]:
path ='/content/drive/MyDrive/BCTP/nimess_features.csv'
nimess_eventlog = pd.read_csv(path, low_memory=False)

In [22]:
path ='/content/drive/MyDrive/BCTP/anino_features.csv'
anino_eventlog = pd.read_csv(path, low_memory=False)

In [23]:
nsml_eventlog = pd.read_csv('/content/drive/MyDrive/BCTP/nsml_eventlog_aggregated.csv', low_memory=False)

In [24]:
path ='/content/drive/MyDrive/BCTP/trfl_eventlog.csv'
trfl_eventlog = pd.read_csv(path, low_memory=False)

In [25]:
path ='/content/drive/MyDrive/BCTP/xgboost_data.csv'
final_df = pd.read_csv(path, low_memory=False)

Data cleaning and preprocessing

In [26]:
nimess_eventlog.drop(['Unnamed: 0'], axis=1, inplace=True)
anino_eventlog.drop(['Unnamed: 0'], axis=1, inplace=True)
nsml_eventlog.drop(['Unnamed: 0'], axis=1, inplace=True)
trfl_eventlog.drop(['Unnamed: 0'], axis=1, inplace=True)
final_df.drop(['Unnamed: 0'], axis=1, inplace=True)

nimess_eventlog['date_column'] = pd.to_datetime(nimess_eventlog['date_column'])
anino_eventlog['date_column'] = pd.to_datetime(anino_eventlog['date_column'])
nsml_eventlog['date_column'] = pd.to_datetime(nsml_eventlog['date_column'])
trfl_eventlog['date_column'] = pd.to_datetime(trfl_eventlog['date_column'])
final_df['date_column'] = pd.to_datetime(final_df['date_column'])

nimess_eventlog['Gender'] = nimess_eventlog['kjønn'].map({'Mann': 1, 'Kvinne': 0})

nimess_eventlog.drop(['index', 'disease_groups', 'kjønn', 'prior_comorbidities', 'inndatotid', 'utdatotid'], axis=1, inplace=True)

In [27]:
# Calculate the number of rows for each 'ppid'
ppid_row_counts = nimess_eventlog['ppid'].value_counts()

# Bin these counts into specified ranges
bins = [0, 3, 6, 301, 601, 1001, float('inf')]  # Adjusted bins to include the edge values
labels = ['0-2', '3-5', '6-300', '301-600', '601-1000', '1000+']
binned_counts = pd.cut(ppid_row_counts, bins=bins, labels=labels, right=False).value_counts().sort_index()

# Display the binned counts to verify
print(binned_counts)


count
0-2          2777
3-5          3497
6-300       29082
301-600       179
601-1000       42
1000+          14
Name: count, dtype: int64


In [28]:
# Calculate the number of rows for each 'ppid'
ppid_row_counts = trfl_eventlog['ppid'].value_counts()

# Bin these counts into specified ranges
bins = [0, 3, 6, 201, 301, 1001, float('inf')]  # Adjusted bins to include the edge values
labels = ['0-2', '3-5', '6-100', '201-300', '301-1000', '1000+']
binned_counts = pd.cut(ppid_row_counts, bins=bins, labels=labels, right=False).value_counts().sort_index()

# Display the binned counts to verify
print(binned_counts)


count
0-2          2473
3-5          4732
6-100       27122
201-300        93
301-1000       17
1000+           0
Name: count, dtype: int64


In [29]:
final_df

,blood_culture_test_encoded,date_column,ppid,ppid.1,urgency_code,care_level_code,LOS,time_to_last,total_los,age,...,prior_positive_faeces,prior_positive_hal,prior_positive_hud,prior_positive_led,prior_positive_naso,prior_positive_plasma,prior_positive_tunge,prior_positive_urin,prior_positive_ear,prior_positive_eye
0,0.0,2014-05-08,1.0,1,1,1,5.458333,1524.0,22.958333,81,...,0,0,0,0,0,0,0,0,0,0
1,0.0,2014-08-31,1.0,1,1,1,4.958333,456.0,27.916667,81,...,0,0,0,0,0,0,0,0,0,0
2,0.0,2014-12-29,1.0,1,1,1,1.500000,838.0,29.416667,81,...,0,0,0,0,0,0,0,0,0,0
3,0.0,2013-10-28,2.0,2,1,1,3.000000,0.0,3.000000,44,...,0,0,0,0,0,0,0,0,0,0
4,0.0,2019-04-29,3.0,3,4,1,10.208333,65.0,131.166667,64,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65970,0.0,2017-05-30,36351.0,36351,1,1,13.833333,9.0,31.750000,61,...,0,0,0,0,0,0,0,1,0,0
65971,0.0,2017-05-31,36351.0,36351,1,1,13.833333,9.0,31.750000,61,...,0,0,0,0,0,0,0,1,0,0
65972,0.0,2017-06-01,36351.0,36351,1,1,13.833333,9.0,31.750000,61,...,0,0,0,0,0,0,0,1,0,0
65973,0.0,2017-06-02,36351.0,36351,1,1,13.833333,9.0,31.750000,61,...,0,0,0,0,0,0,0,1,0,0


In [31]:
data = final_df

In [32]:
import pandas as pd
from sklearn.utils import resample

# Separate the majority (negative) and minority (positive) classes
negative_cases = data[data['blood_culture_test_encoded'] == 0.0]
positive_cases = data[data['blood_culture_test_encoded'] == 1.0]

# Ensure each patient has only one test by dropping duplicates based on patient ID (assuming 'ppid' is the patient ID column)
negative_cases_unique = negative_cases.drop_duplicates(subset='ppid')
positive_cases_unique = positive_cases.drop_duplicates(subset='ppid')

# Downsample the cases to get 500 unique patients/tests in each class
negative_downsampled = resample(negative_cases_unique,
                                replace=False,  # sample without replacement
                                n_samples=500,  # number of samples to match
                                random_state=42)  # reproducibility

positive_downsampled = resample(positive_cases_unique,
                                replace=False,  # sample without replacement
                                n_samples=500,  # number of samples to match
                                random_state=42)  # reproducibility

# Combine the downsampled negative and positive cases
balanced_data = pd.concat([negative_downsampled, positive_downsampled])

# Shuffle the resulting dataset to mix positive and negative cases
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the balanced dataset to a new CSV file
balanced_data.to_csv('balanced_xgboost_data.csv', index=False)


In [33]:
balanced_data

,blood_culture_test_encoded,date_column,ppid,ppid.1,urgency_code,care_level_code,LOS,time_to_last,total_los,age,...,prior_positive_faeces,prior_positive_hal,prior_positive_hud,prior_positive_led,prior_positive_naso,prior_positive_plasma,prior_positive_tunge,prior_positive_urin,prior_positive_ear,prior_positive_eye
0,1.0,2016-05-29,10490.0,10490,1,1,23.208333,122.0,28.291667,87,...,0,0,0,0,0,0,0,0,0,0
1,1.0,2018-05-18,22294.0,22294,1,1,4.541667,25848.0,4.541667,67,...,0,0,0,0,0,0,0,0,0,0
2,1.0,2017-07-30,17001.0,17001,1,1,9.958333,11692.0,59.958333,87,...,0,0,0,0,0,0,0,11,0,0
3,1.0,2018-12-11,11363.0,11363,1,1,4.916667,6030.0,5.125000,39,...,0,0,0,0,0,0,0,0,0,0
4,0.0,2018-04-09,21710.0,21710,4,1,3.958333,503.0,44.625000,70,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,2015-08-04,27763.0,27763,4,1,9.208333,665.0,69.916667,74,...,0,0,0,0,0,0,0,0,0,0
996,0.0,2015-12-04,33210.0,33210,1,1,3.750000,8960.0,12.416667,71,...,0,0,0,0,0,0,0,0,0,0
997,1.0,2013-04-26,2563.0,2563,1,1,3.916667,0.0,3.916667,60,...,0,0,0,0,0,0,0,1,0,0
998,0.0,2017-05-14,8420.0,8420,1,1,10.916667,7733.0,50.166667,83,...,0,0,0,0,0,0,0,1,0,0


In [34]:
anino_eventlog.columns

Index(['ppid', 'ICU_LOS', 'date_column', 'ICU_LOS_total'], dtype='object')

In [35]:
trfl_eventlog.columns

Index(['ppid', 'BILIRUBIN KONJUGERT', 'BILIRUBIN TOTAL',
       'BILIRUBIN UKONJUGERT', 'CRP', 'CRP-HØYSENSITIV', 'KREATININ', 'LAKTAT',
       'LAKTAT BLODGASS', 'LAKTAT BLODGASS VENØST', 'LAKTAT PNA',
       'LEUKOCYTTER', 'PH', 'PH PNA', 'PO2', 'PO2 PNA', 'TROMBOCYTTER',
       'date_column'],
      dtype='object')

In [36]:
nimess_eventlog.columns

Index(['ppid', 'urgency_code', 'care_level_code', 'LOS', 'time_to_last',
       'total_los', 'age', 'procedure_0', 'procedure_1', 'procedure_2',
       ...
       'procedure_R_aggregate', 'procedure_S_aggregate',
       'procedure_T_aggregate', 'procedure_U_aggregate',
       'procedure_V_aggregate', 'procedure_W_aggregate',
       'procedure_X_aggregate', 'procedure_Y_aggregate',
       'procedure_Z_aggregate', 'Gender'],
      dtype='object', length=167)

In [37]:
def encode_test_results(nsml_eventlog):
    test_result_columns = [col for col in nsml_eventlog.columns if col not in ['ppid', 'date_column']]

    for col in test_result_columns:
        # Ensure the column data is in string format
        nsml_eventlog[col] = nsml_eventlog[col].astype(str)

        nsml_eventlog[col] = nsml_eventlog[col].apply(
            lambda x: 5 if 'positive' in x else
                      2 if 'contaminant' in x and 'positive' not in x else
                      1 if 'negative' in x and 'positive' not in x and 'contaminant' not in x else
                      0  # Default value for empty or unspecified case
        )
    return nsml_eventlog

# Apply the updated encoding logic
encoded_nsml_eventlog = encode_test_results(nsml_eventlog.copy())


In [38]:
nimess_eventlog = nimess_eventlog.fillna(0)
anino_eventlog = anino_eventlog.fillna(0)
trfl_eventlog = trfl_eventlog.fillna(0)
encoded_nsml_eventlog = encoded_nsml_eventlog.fillna(0)

nimess_eventlog = nimess_eventlog[['ppid',
       'urgency_code', 'care_level_code', 'LOS',
       'time_to_last', 'total_los',
       'Gender', 'age', 'procedure_0', 'procedure_1',
       'procedure_3', 'procedure_4', 'procedure_A', 'procedure_B',
       'procedure_C', 'procedure_D', 'procedure_E', 'procedure_F',
       'procedure_G', 'procedure_H', 'procedure_I', 'procedure_J',
       'procedure_K', 'procedure_L', 'procedure_M', 'procedure_N',
       'procedure_O', 'procedure_P', 'procedure_Q', 'procedure_R',
       'procedure_T', 'procedure_U', 'procedure_W', 'procedure_X',
       'procedure_Z', 'ICD_A', 'ICD_B', 'ICD_C', 'ICD_D', 'ICD_E', 'ICD_F',
       'ICD_G', 'ICD_H', 'ICD_I', 'ICD_J', 'ICD_K', 'ICD_L', 'ICD_M', 'ICD_N',
       'ICD_O', 'ICD_Q', 'ICD_R', 'ICD_S', 'ICD_T', 'ICD_U', 'ICD_V', 'ICD_W',
       'ICD_X', 'ICD_Y', 'ICD_Z', 'date_column', 'urinarytractinfection', 'cardiovascular', 'lung', 'centralnervoussystem', 'organdysfunction', 'skinandsofttissueinfection',
                                   'pneumonia', 'endocarditis', 'sepsis', 'infection', 'dementia', 'explicitsepsis', 'intraabdominalinfection', 'cancer']]

balanced_data.drop(['date_column.2', 'ppid.2', 'date_column.1', 'ppid.1'], axis=1, inplace=True)

In [39]:
encoded_nsml_eventlog

,ppid,date_column,annet,anus,bein,biopsi,blod,blood_culture_test,bronki,melk,...,faeces,hal,hud,led,naso,plasma,tunge,urin,ear,eye
0,1,2013-10-08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,2014-05-08,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2014-08-31,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2014-09-01,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,2014-12-29,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273468,36351,2017-06-01,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
273469,36351,2017-06-02,0,0,0,0,0,1,0,0,...,0,5,0,0,0,0,0,0,0,0
273470,36351,2017-06-07,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
273471,36351,2017-06-27,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
final_df

,blood_culture_test_encoded,date_column,ppid,urgency_code,care_level_code,LOS,time_to_last,total_los,age,procedure_0,...,prior_positive_faeces,prior_positive_hal,prior_positive_hud,prior_positive_led,prior_positive_naso,prior_positive_plasma,prior_positive_tunge,prior_positive_urin,prior_positive_ear,prior_positive_eye
0,0.0,2014-05-08,1.0,1,1,5.458333,1524.0,22.958333,81,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,2014-08-31,1.0,1,1,4.958333,456.0,27.916667,81,0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,2014-12-29,1.0,1,1,1.500000,838.0,29.416667,81,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,2013-10-28,2.0,1,1,3.000000,0.0,3.000000,44,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,2019-04-29,3.0,4,1,10.208333,65.0,131.166667,64,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65970,0.0,2017-05-30,36351.0,1,1,13.833333,9.0,31.750000,61,0,...,0,0,0,0,0,0,0,1,0,0
65971,0.0,2017-05-31,36351.0,1,1,13.833333,9.0,31.750000,61,0,...,0,0,0,0,0,0,0,1,0,0
65972,0.0,2017-06-01,36351.0,1,1,13.833333,9.0,31.750000,61,0,...,0,0,0,0,0,0,0,1,0,0
65973,0.0,2017-06-02,36351.0,1,1,13.833333,9.0,31.750000,61,0,...,0,0,0,0,0,0,0,1,0,0


For every blood culture test creating sequences of medical events leading upto it from the four eventlogs (nimess_eventlog, anino_eventlog, trfl_eventlog, nsml_eventlog)

In [40]:
# Function to create and transform a sequence into a numeric numpy array
def transform_sequence(ppid, date_req, nimess_eventlog, anino_eventlog, trfl_eventlog, encoded_nsml_eventlog):
    # Assuming create_sequences function is already defined and it combines the data correctly
    sequence, feature_names = create_sequences(ppid, date_req, nimess_eventlog, anino_eventlog, trfl_eventlog, encoded_nsml_eventlog)

    # Convert dataframe to numpy array
    numeric_array = sequence.to_numpy()
    return numeric_array, feature_names

In [41]:
def create_sequences(patient_id, test_date, nimess_eventlog, anino_eventlog, trfl_eventlog, nsml_eventlog, use_full_history=False, window_size=1095):
    # Define the start date for the sequence
    if use_full_history:
        start_date = pd.Timestamp.min
    else:
        start_date = test_date - pd.Timedelta(days=window_size)

    # Filter each dataframe for the given patient and time window
    nsml_events = nsml_eventlog[(nsml_eventlog['ppid'] == patient_id) &
                                (nsml_eventlog['date_column'] < test_date) &
                                (nsml_eventlog['date_column'] >= start_date)]

    trfl_events = trfl_eventlog[(trfl_eventlog['ppid'] == patient_id) &
                                (trfl_eventlog['date_column'] <= test_date) &
                                (trfl_eventlog['date_column'] >= start_date)]

    nimess_events = nimess_eventlog[(nimess_eventlog['ppid'] == patient_id) &
                                    (nimess_eventlog['date_column'] <= test_date) &
                                    (nimess_eventlog['date_column'] >= start_date)]

    anino_events = anino_eventlog[(anino_eventlog['ppid'] == patient_id) &
                                  (anino_eventlog['date_column'] <= test_date) &
                                  (anino_eventlog['date_column'] >= start_date)]

    # Merge the dataframes using outer join
    merged_df = nsml_events.merge(trfl_events, on=['ppid', 'date_column'], how='outer')
    merged_df = merged_df.merge(nimess_events, on=['ppid', 'date_column'], how='outer')
    merged_df = merged_df.merge(anino_events, on=['ppid', 'date_column'], how='outer')

    # Sort by date_column
    merged_df = merged_df.sort_values(by='date_column')

    # Remove 'date_column' and 'ppid' columns
    merged_df = merged_df.drop(columns=['date_column', 'ppid'])

    feature_names = merged_df.columns.tolist()

    # Additional data processing can be done here

    return merged_df, feature_names

In [42]:
# Initialize empty lists for sequences and labels
sequences = []
labels = []
feature_names_list = []  # To store feature names
# Define batch size
batch_size = 10000  # Adjust this based on your dataset and available memory

In [43]:
def process_batch(batch_df):
    batch_data = batch_df.apply(
        lambda row: transform_sequence(row['ppid'], row['date_column'], nimess_eventlog, anino_eventlog, trfl_eventlog, encoded_nsml_eventlog),
        axis=1
    )

    sequences_batch, feature_names_batch = zip(*batch_data)

    labels_batch = batch_df['blood_culture_test_encoded'].tolist()

    # Ensure all feature names are the same across batches
    assert all(feature_names_batch[0] == fn for fn in feature_names_batch), "Inconsistent feature names across batches"

    return sequences_batch, labels_batch, feature_names_batch[0]  # Returning the first batch's feature names


# Process in batches
for start in range(0, len(balanced_data), batch_size):
    end = min(start + batch_size, len(balanced_data))
    batch_df = balanced_data.iloc[start:end]
    sequences_batch, labels_batch, feature_names = process_batch(batch_df)
    sequences.extend(sequences_batch)
    labels.extend(labels_batch)
    if not feature_names_list:  # If feature_names_list is empty, populate it
        feature_names_list.extend(feature_names)

    print(f"Processed batch {start//batch_size + 1}/{len(balanced_data)//batch_size + 1}")


Processed batch 1/1


In [44]:
# Determine the length to pad/truncate to based on the longest sequence
max_sequence_length = max(len(sequence) for sequence in sequences)
max_sequence_length

542

In [45]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [46]:
sequence_length = min(max_sequence_length, 500)  # Setting an upper limit

In [47]:
# Pad or truncate sequences in batches
padded_sequences = []
padding_batch_size = 1000  # Adjust based on available memory

for start in range(0, len(sequences), padding_batch_size):
    end = min(start + padding_batch_size, len(sequences))
    sequences_batch = sequences[start:end]
    padded_batch = pad_sequences(sequences_batch, maxlen=sequence_length, padding='pre', truncating='pre', dtype='float32')
    padded_sequences.extend(padded_batch)

    print(f"Padding batch {start//padding_batch_size + 1}/{len(sequences)//padding_batch_size + 1}")

# Convert the padded sequences list to a numpy array
padded_sequences = np.array(padded_sequences)

# Convert labels to a numpy array
labels = np.array(labels)


# Inspect shapes and connections
print("Shape of sequences:", padded_sequences.shape)
print("Shape of labels:", labels.shape)

Padding batch 1/2
Shape of sequences: (1000, 500, 110)
Shape of labels: (1000,)


In [48]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [49]:
# Calculate the sequence length (if not already defined)
sequence_length = padded_sequences.shape[1]

# Generate flattened feature names
flattened_feature_names = []
for t in range(sequence_length):
    for feature_name in feature_names_list:
        flattened_feature_names.append(f"{feature_name}_t{t+1}")

In [50]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weight_dict = dict(enumerate(class_weights))

print("Class weights:", class_weight_dict)


Class weights: {0: 1.0, 1: 1.0}


In [51]:
from sklearn.model_selection import train_test_split

# Initial train-test split with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences,
    labels,
    test_size=0.2,
    random_state=None,
    shuffle=False)  # Notice the 'stratify' parameter here

# Further split the training data into a smaller train set and a validation set, also with stratified sampling
X_train_trans, X_val_trans, y_train_trans, y_val_trans = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=None,
    shuffle=True)  # And also here

# Now, X_train_lstm, X_val_lstm, and X_test should have the same class proportions as the original labels array

In [52]:
from sklearn.preprocessing import StandardScaler

# Replace NaN values with 0
X_train_trans = np.nan_to_num(X_train_trans)
X_val_trans = np.nan_to_num(X_val_trans)
X_test = np.nan_to_num(X_test)

# Store original shapes for training, validation, and test sets
original_shape_train = X_train_trans.shape  # (num_samples, timesteps, features)
original_shape_val = X_val_trans.shape  # Validation set shape
original_shape_test = X_test.shape  # Test set shape

# Flatten the data for scaling (reshape to 2D)
X_train_reshaped = X_train_trans.reshape(-1, X_train_trans.shape[-1])
X_val_reshaped = X_val_trans.reshape(-1, X_val_trans.shape[-1])
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

# Initialize a StandardScaler and fit it on X_train_reshaped
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped)

# Apply the same transformation to X_val_reshaped and X_test_reshaped
X_val_scaled = scaler.transform(X_val_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

# Reshape back to original shape
X_train_trans = X_train_scaled.reshape(original_shape_train)
X_val_trans = X_val_scaled.reshape(original_shape_val)
X_test = X_test_scaled.reshape(original_shape_test)

In [53]:
print(X_train_trans.shape, y_train_trans.shape)

(680, 500, 110) (680,)


Adding anomaly scores for the transformer model

In [54]:
if np.isnan(X_train_trans).any():
    print("NaN values present in the original data")
else:
    print("No NaN values in the original data")

# Assuming X_train is your 3D training data (samples, timesteps, features)
# Flatten or aggregate your 3D data (X_train) into 2D
X_train_2d = np.mean(X_train, axis=1)  # Mean aggregation

# Verify that the transformation was successful
print(f"Shape of X_train_2d: {X_train_2d.shape}")
print(f"Shape of y_train: {y_train_trans.shape}")
# After transformation, check if NaN values are introduced
if np.isnan(X_train_2d).any():
    print("NaN values detected after transformation")

No NaN values in the original data
Shape of X_train_2d: (800, 110)
Shape of y_train: (680,)
NaN values detected after transformation


In [55]:
# Fill NaN values in the transformed data, if they exist
X_train_2d = np.nan_to_num(X_train_2d)

In [56]:
from sklearn.ensemble import IsolationForest

# Filter out the normal data for training the anomaly detection model
normal_data_2d = X_train_2d[y_train == 0]

# Train the Isolation Forest model
iso_forest = IsolationForest(n_estimators=100, random_state=42, contamination='auto')
iso_forest.fit(normal_data_2d)

IsolationForest(random_state=42)

In [57]:
# Transform each subset from 3D to 2D
X_train_trans_2d = np.mean(X_train_trans, axis=1)
X_val_trans_2d = np.mean(X_val_trans, axis=1)
X_test_2d = np.mean(X_test, axis=1)
# Fill NaN values in the transformed data, if they exist
X_train_trans_2d = np.nan_to_num(X_train_trans_2d)
X_val_trans_2d = np.nan_to_num(X_val_trans_2d)
X_test_2d = np.nan_to_num(X_test_2d)

# Compute anomaly scores for each subset
anomaly_scores_train_trans = iso_forest.decision_function(X_train_trans_2d)
anomaly_scores_val_trans = iso_forest.decision_function(X_val_trans_2d)
anomaly_scores_test = iso_forest.decision_function(X_test_2d)

# Normalize the anomaly scores
anomaly_scores_train_norm = np.interp(anomaly_scores_train_trans, (anomaly_scores_train_trans.min(), anomaly_scores_train_trans.max()), (0, 1))
anomaly_scores_val_norm = np.interp(anomaly_scores_val_trans, (anomaly_scores_val_trans.min(), anomaly_scores_val_trans.max()), (0, 1))
anomaly_scores_test_norm = np.interp(anomaly_scores_test, (anomaly_scores_test.min(), anomaly_scores_test.max()), (0, 1))


In [58]:
# Function to expand and append anomaly scores to original data
def append_anomaly_scores(original_data, scores):
    # Expanding the scores to match the original data shape
    scores_expanded = np.expand_dims(scores, axis=-1)  # Adding an extra dimension
    scores_expanded = np.repeat(scores_expanded, original_data.shape[1], axis=1)  # Repeat across timesteps
    scores_expanded = np.expand_dims(scores_expanded, axis=-1)  # Adding another dimension for features

    return np.concatenate((original_data, scores_expanded), axis=2)  # Concatenate along the feature axis

# Apply the function to each subset
X_train_trans_extended = append_anomaly_scores(X_train_trans, anomaly_scores_train_norm)
X_val_trans_extended = append_anomaly_scores(X_val_trans, anomaly_scores_val_norm)
X_test_extended = append_anomaly_scores(X_test, anomaly_scores_test_norm)

In [59]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, auc, precision_recall_curve

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_class = (y_pred > 0.5).astype("int32")

    accuracy = accuracy_score(y_test, y_pred_class)
    precision = precision_score(y_test, y_pred_class)
    recall = recall_score(y_test, y_pred_class)
    f1 = f1_score(y_test, y_pred_class)
    roc_auc = roc_auc_score(y_test, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_class).ravel()
    specificity = tn / (tn + fp)

    # AUPRC calculation
    precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred)
    auprc = auc(recall_curve, precision_curve)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"AUPRC Score: {auprc:.4f}")

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix
import numpy as np

def evaluate_model2(model, X_test, y_test):
    # Predict the probabilities
    y_pred_prob = model.predict(X_test)

    # Convert probabilities to class predictions
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate precision
    precision = precision_score(y_true, y_pred, average='binary')

    # Calculate recall (sensitivity)
    recall = recall_score(y_true, y_pred, average='binary')

    # Calculate specificity
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)

    # Calculate F1 score
    f1 = f1_score(y_true, y_pred, average='binary')

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')

    # Calculate AUPRC score
    auprc = average_precision_score(y_test, y_pred_prob)

    # Print the metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"AUPRC Score: {auprc:.4f}")


In [60]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [61]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input
from tensorflow.keras.layers import MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam

In [62]:
# Set the learning rate
learning_rate = 0.0001

# Transformer block as a function
def transformer_block(inputs, num_heads, ff_dim, dropout=0.2):
    # Multi-head attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(inputs, inputs)

    # Skip connection and normalization
    attention_output = Dropout(dropout)(attention_output)
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attention_output)

    # Feed-forward layer
    ffn_output = Dense(ff_dim, activation='relu')(out1)
    ffn_output = Dropout(dropout)(ffn_output)
    ffn_output = Dense(inputs.shape[-1])(ffn_output)

    # Second skip connection and normalization
    out2 = LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

    return out2

# Model definition
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
inputs = Input(shape=input_shape)
x = inputs

# Create Transformer layers
for _ in range(2):  # Number of Transformer blocks
    x = transformer_block(x, num_heads=4, ff_dim=512)

# Output layers
x = GlobalAveragePooling1D()(x)
x = Dropout(0.2)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train_trans_extended, y_train_trans,
    epochs=10,
    batch_size=128,
    validation_data=(X_val_trans_extended, y_val_trans),
    class_weight=class_weight_dict
)

Epoch 1/10
6/6 [==============================] - 66s 10s/step - loss: 0.8311 - accuracy: 0.5044 - val_loss: 0.6957 - val_accuracy: 0.4917
Epoch 2/10
6/6 [==============================] - 61s 10s/step - loss: 0.6884 - accuracy: 0.5279 - val_loss: 0.6918 - val_accuracy: 0.5250
Epoch 3/10
6/6 [==============================] - 61s 10s/step - loss: 0.6879 - accuracy: 0.5324 - val_loss: 0.6786 - val_accuracy: 0.5167
Epoch 4/10
6/6 [==============================] - 61s 10s/step - loss: 0.6732 - accuracy: 0.5691 - val_loss: 0.6712 - val_accuracy: 0.6917
Epoch 5/10
6/6 [==============================] - 61s 10s/step - loss: 0.6693 - accuracy: 0.5809 - val_loss: 0.6646 - val_accuracy: 0.6500
Epoch 6/10
6/6 [==============================] - 62s 10s/step - loss: 0.6563 - accuracy: 0.5956 - val_loss: 0.6596 - val_accuracy: 0.6500
Epoch 7/10
6/6 [==============================] - 61s 10s/step - loss: 0.6427 - accuracy: 0.6235 - val_loss: 0.6643 - val_accuracy: 0.6667
Epoch 8/10
6/6 [===========

In [63]:
print("Evaluating Transformer Model")
evaluate_model(model, X_test_extended, y_test)
#print("Evaluating Transformer Model")
#evaluate_model2(model, X_test_extended, y_test)

Evaluating Transformer Model
7/7 [==============================] - 8s 1s/step
Accuracy: 0.5900
Precision: 0.6545
Recall (Sensitivity): 0.3636
Specificity: 0.8119
F1 Score: 0.4675
ROC AUC Score: 0.6050
AUPRC Score: 0.6294


In [64]:
from tensorflow.keras.layers import GRU, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten
# CNN-GRU Model
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
inputs = Input(shape=input_shape)
x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = GRU(64)(x)
x = Dense(20, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

cnn_gru_model = Model(inputs=inputs, outputs=outputs)
cnn_gru_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
cnn_gru_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=128, validation_data=(X_val_trans_extended, y_val_trans), class_weight=class_weight_dict)


Epoch 1/10
6/6 [==============================] - 4s 298ms/step - loss: 0.7276 - accuracy: 0.5250 - val_loss: 0.7409 - val_accuracy: 0.4833
Epoch 2/10
6/6 [==============================] - 1s 199ms/step - loss: 0.6978 - accuracy: 0.5338 - val_loss: 0.7247 - val_accuracy: 0.5000
Epoch 3/10
6/6 [==============================] - 1s 194ms/step - loss: 0.6786 - accuracy: 0.5735 - val_loss: 0.7164 - val_accuracy: 0.5167
Epoch 4/10
6/6 [==============================] - 1s 208ms/step - loss: 0.6670 - accuracy: 0.5941 - val_loss: 0.7114 - val_accuracy: 0.5417
Epoch 5/10
6/6 [==============================] - 1s 190ms/step - loss: 0.6540 - accuracy: 0.6221 - val_loss: 0.7069 - val_accuracy: 0.5333
Epoch 6/10
6/6 [==============================] - 1s 190ms/step - loss: 0.6424 - accuracy: 0.6412 - val_loss: 0.7024 - val_accuracy: 0.5250
Epoch 7/10
6/6 [==============================] - 1s 190ms/step - loss: 0.6306 - accuracy: 0.6529 - val_loss: 0.6984 - val_accuracy: 0.5417
Epoch 8/10
6/6 [====

In [65]:
print("Evaluating CNN-GRU Model")
evaluate_model(cnn_gru_model, X_test_extended, y_test)

Evaluating CNN-GRU Model
7/7 [==============================] - 1s 22ms/step
Accuracy: 0.6300
Precision: 0.6437
Recall (Sensitivity): 0.5657
Specificity: 0.6931
F1 Score: 0.6022
ROC AUC Score: 0.6624
AUPRC Score: 0.6242


In [66]:
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

# CNN-LSTM Model
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
inputs = Input(shape=input_shape)
x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = LSTM(64)(x)
x = Dense(20, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

cnn_lstm_model = Model(inputs=inputs, outputs=outputs)
cnn_lstm_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
cnn_lstm_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=128, validation_data=(X_val_trans_extended, y_val_trans), class_weight=class_weight_dict)


Epoch 1/10
6/6 [==============================] - 4s 329ms/step - loss: 0.7375 - accuracy: 0.4941 - val_loss: 0.6911 - val_accuracy: 0.5000
Epoch 2/10
6/6 [==============================] - 1s 219ms/step - loss: 0.7117 - accuracy: 0.5088 - val_loss: 0.6831 - val_accuracy: 0.5333
Epoch 3/10
6/6 [==============================] - 1s 222ms/step - loss: 0.6947 - accuracy: 0.5191 - val_loss: 0.6786 - val_accuracy: 0.5667
Epoch 4/10
6/6 [==============================] - 1s 224ms/step - loss: 0.6832 - accuracy: 0.5426 - val_loss: 0.6744 - val_accuracy: 0.5833
Epoch 5/10
6/6 [==============================] - 1s 219ms/step - loss: 0.6723 - accuracy: 0.5706 - val_loss: 0.6713 - val_accuracy: 0.6333
Epoch 6/10
6/6 [==============================] - 1s 215ms/step - loss: 0.6632 - accuracy: 0.6000 - val_loss: 0.6688 - val_accuracy: 0.6333
Epoch 7/10
6/6 [==============================] - 1s 223ms/step - loss: 0.6543 - accuracy: 0.6221 - val_loss: 0.6660 - val_accuracy: 0.6417
Epoch 8/10
6/6 [====

In [67]:
print("Evaluating CNN-LSTM Model")
evaluate_model(cnn_lstm_model, X_test_extended, y_test)

Evaluating CNN-LSTM Model
7/7 [==============================] - 1s 31ms/step
Accuracy: 0.5600
Precision: 0.5390
Recall (Sensitivity): 0.7677
Specificity: 0.3564
F1 Score: 0.6333
ROC AUC Score: 0.6416
AUPRC Score: 0.6344


In [68]:
from tensorflow.keras.layers import Input, GRU, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# GRU Model
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
inputs = Input(shape=input_shape)
x = GRU(128, return_sequences=True)(inputs)
x = GRU(64)(x)
x = Dense(20, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

gru_model = Model(inputs=inputs, outputs=outputs)
gru_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

gru_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=128, validation_data=(X_val_trans_extended, y_val_trans), class_weight=class_weight_dict)


Epoch 1/10
6/6 [==============================] - 11s 1s/step - loss: 0.7110 - accuracy: 0.4794 - val_loss: 0.7148 - val_accuracy: 0.4333
Epoch 2/10
6/6 [==============================] - 6s 1s/step - loss: 0.6956 - accuracy: 0.5162 - val_loss: 0.7092 - val_accuracy: 0.4333
Epoch 3/10
6/6 [==============================] - 6s 1s/step - loss: 0.6841 - accuracy: 0.5529 - val_loss: 0.7040 - val_accuracy: 0.4333
Epoch 4/10
6/6 [==============================] - 6s 1s/step - loss: 0.6735 - accuracy: 0.5809 - val_loss: 0.6987 - val_accuracy: 0.4583
Epoch 5/10
6/6 [==============================] - 6s 1s/step - loss: 0.6640 - accuracy: 0.6132 - val_loss: 0.6928 - val_accuracy: 0.4667
Epoch 6/10
6/6 [==============================] - 6s 1s/step - loss: 0.6552 - accuracy: 0.6309 - val_loss: 0.6874 - val_accuracy: 0.4667
Epoch 7/10
6/6 [==============================] - 6s 1s/step - loss: 0.6466 - accuracy: 0.6500 - val_loss: 0.6827 - val_accuracy: 0.5000
Epoch 8/10
6/6 [========================

In [69]:
print("Evaluating GRU Model")
evaluate_model(gru_model, X_test_extended, y_test)

Evaluating GRU Model
7/7 [==============================] - 1s 110ms/step
Accuracy: 0.5550
Precision: 0.5446
Recall (Sensitivity): 0.6162
Specificity: 0.4950
F1 Score: 0.5782
ROC AUC Score: 0.5927
AUPRC Score: 0.5699


In [70]:
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# LSTM Model
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
inputs = Input(shape=input_shape)
x = LSTM(128, return_sequences=True)(inputs)
x = LSTM(64)(x)
x = Dense(20, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

lstm_model = Model(inputs=inputs, outputs=outputs)
lstm_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=128, validation_data=(X_val_trans_extended, y_val_trans), class_weight=class_weight_dict)


Epoch 1/10
6/6 [==============================] - 12s 1s/step - loss: 0.6962 - accuracy: 0.4824 - val_loss: 0.6996 - val_accuracy: 0.4250
Epoch 2/10
6/6 [==============================] - 8s 1s/step - loss: 0.6874 - accuracy: 0.5368 - val_loss: 0.6958 - val_accuracy: 0.4833
Epoch 3/10
6/6 [==============================] - 8s 1s/step - loss: 0.6806 - accuracy: 0.5809 - val_loss: 0.6921 - val_accuracy: 0.4750
Epoch 4/10
6/6 [==============================] - 8s 1s/step - loss: 0.6743 - accuracy: 0.6265 - val_loss: 0.6886 - val_accuracy: 0.4833
Epoch 5/10
6/6 [==============================] - 8s 1s/step - loss: 0.6682 - accuracy: 0.6559 - val_loss: 0.6852 - val_accuracy: 0.4750
Epoch 6/10
6/6 [==============================] - 7s 1s/step - loss: 0.6621 - accuracy: 0.6735 - val_loss: 0.6817 - val_accuracy: 0.4917
Epoch 7/10
6/6 [==============================] - 7s 1s/step - loss: 0.6559 - accuracy: 0.6779 - val_loss: 0.6779 - val_accuracy: 0.5167
Epoch 8/10
6/6 [========================

In [71]:
print("Evaluating LSTM Model")
evaluate_model(lstm_model, X_test_extended, y_test)

Evaluating LSTM Model
7/7 [==============================] - 2s 199ms/step
Accuracy: 0.5700
Precision: 0.5607
Recall (Sensitivity): 0.6061
Specificity: 0.5347
F1 Score: 0.5825
ROC AUC Score: 0.5919
AUPRC Score: 0.5987


In [72]:
from tensorflow.keras.utils import to_categorical

# Ensure the labels are one-hot encoded
y_train_trans = to_categorical(y_train_trans, num_classes=2)
y_val_trans = to_categorical(y_val_trans, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Now, define and train the DKN model again
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Flatten
from tensorflow.keras.models import Model

# Define the DKN model
def create_dkn_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # LSTM layer
    lstm_out = LSTM(128, return_sequences=True)(inputs)

    # Flatten the LSTM output
    lstm_out_flattened = Flatten()(lstm_out)

    # Dense layers for knowledge-aware processing
    dense_out = Dense(128, activation='relu')(lstm_out_flattened)

    # Output layer
    outputs = Dense(num_classes, activation='softmax')(dense_out)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Define the input shape and number of classes
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
num_classes = 2  # Assuming binary classification

# Create the model
dkn_model = create_dkn_model(input_shape, num_classes)

# Train the model
dkn_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=32, validation_data=(X_val_trans_extended, y_val_trans))


Epoch 1/10
22/22 [==============================] - 11s 427ms/step - loss: 1.1940 - accuracy: 0.5338 - val_loss: 0.7187 - val_accuracy: 0.6500
Epoch 2/10
22/22 [==============================] - 9s 412ms/step - loss: 0.7460 - accuracy: 0.7338 - val_loss: 0.7027 - val_accuracy: 0.6083
Epoch 3/10
22/22 [==============================] - 9s 424ms/step - loss: 0.3315 - accuracy: 0.8676 - val_loss: 0.7347 - val_accuracy: 0.6583
Epoch 4/10
22/22 [==============================] - 9s 423ms/step - loss: 0.2186 - accuracy: 0.9265 - val_loss: 0.8264 - val_accuracy: 0.6917
Epoch 5/10
22/22 [==============================] - 9s 409ms/step - loss: 0.1660 - accuracy: 0.9353 - val_loss: 0.9236 - val_accuracy: 0.5917
Epoch 6/10
22/22 [==============================] - 9s 407ms/step - loss: 0.1113 - accuracy: 0.9676 - val_loss: 0.9905 - val_accuracy: 0.5833
Epoch 7/10
22/22 [==============================] - 9s 415ms/step - loss: 0.0760 - accuracy: 0.9735 - val_loss: 1.0742 - val_accuracy: 0.6083
Epoch

In [73]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix
import numpy as np

def evaluate_model2(model, X_test, y_test):
    # Predict the probabilities
    y_pred_prob = model.predict(X_test)

    # Convert probabilities to class predictions
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate precision
    precision = precision_score(y_true, y_pred, average='binary')

    # Calculate recall (sensitivity)
    recall = recall_score(y_true, y_pred, average='binary')

    # Calculate specificity
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)

    # Calculate F1 score
    f1 = f1_score(y_true, y_pred, average='binary')

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')

    # Calculate AUPRC score
    auprc = average_precision_score(y_test, y_pred_prob)

    # Print the metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"AUPRC Score: {auprc:.4f}")


In [74]:
# Evaluate the model
evaluate_model2(dkn_model, X_test_extended, y_test)

7/7 [==============================] - 1s 134ms/step
Accuracy: 0.6250
Precision: 0.6277
Recall (Sensitivity): 0.5960
Specificity: 0.6535
F1 Score: 0.6114
ROC AUC Score: 0.6392
AUPRC Score: 0.6295


In [75]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the CapMatch model
def create_capmatch_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Transformer block
    attn_output = MultiHeadAttention(num_heads=8, key_dim=input_shape[1])(inputs, inputs)
    attn_output = LayerNormalization(epsilon=1e-6)(attn_output)
    attn_output = Dropout(0.1)(attn_output)

    # Flatten the attention output
    attn_output_flattened = Flatten()(attn_output)

    # Capsule network block (simplified version)
    caps_output = Dense(128, activation='relu')(attn_output_flattened)

    # Output layer
    outputs = Dense(num_classes, activation='softmax')(caps_output)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Assuming input_shape is (timesteps, features)
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
num_classes = 2  # Assuming binary classification

# Create the model
capmatch_model = create_capmatch_model(input_shape, num_classes)

# Train the model
capmatch_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=32, validation_data=(X_val_trans_extended, y_val_trans))

# Evaluate the model
evaluate_model2(capmatch_model, X_test_extended, y_test)


Epoch 1/10
22/22 [==============================] - 32s 1s/step - loss: 10.0224 - accuracy: 0.4897 - val_loss: 1.4965 - val_accuracy: 0.5000
Epoch 2/10
22/22 [==============================] - 30s 1s/step - loss: 0.9796 - accuracy: 0.5103 - val_loss: 0.6745 - val_accuracy: 0.5167
Epoch 3/10
22/22 [==============================] - 30s 1s/step - loss: 1.5612 - accuracy: 0.5059 - val_loss: 0.6761 - val_accuracy: 0.5167
Epoch 4/10
22/22 [==============================] - 30s 1s/step - loss: 0.6858 - accuracy: 0.5529 - val_loss: 0.6583 - val_accuracy: 0.5750
Epoch 5/10
22/22 [==============================] - 30s 1s/step - loss: 0.6691 - accuracy: 0.5603 - val_loss: 0.6767 - val_accuracy: 0.5083
Epoch 6/10
22/22 [==============================] - 30s 1s/step - loss: 0.6438 - accuracy: 0.5838 - val_loss: 0.6606 - val_accuracy: 0.5917
Epoch 7/10
22/22 [==============================] - 30s 1s/step - loss: 0.6450 - accuracy: 0.5926 - val_loss: 0.6780 - val_accuracy: 0.5083
Epoch 8/10
22/22 [=

In [76]:
evaluate_model2(capmatch_model, X_test_extended, y_test)

7/7 [==============================] - 4s 551ms/step
Accuracy: 0.5450
Precision: 0.7857
Recall (Sensitivity): 0.1111
Specificity: 0.9703
F1 Score: 0.1947
ROC AUC Score: 0.5500
AUPRC Score: 0.5371
